sentiment analysis

https://spraakbanken.gu.se/resurser/absabank-imm

In [ ]:
#!pip install scikit-learn==1.0.1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.linear_model import SGDClassifier



In [ ]:
#@title
#!wget https://svn.spraakdata.gu.se/sb-arkiv/pub/imm_absabank/datasplit.zip

In [ ]:
!unzip /content/drive/MyDrive/datasplit_sent.zip

In [ ]:
#@title
import os
import pandas as pd

def readDir(pathToDir, split = "train"):
    listOfFiles = []
    for filPath in os.listdir(pathToDir):

        if split in filPath:
          if filPath.endswith(".tsv"):
              listOfFiles.append(os.path.join(pathToDir,filPath))
    return listOfFiles


In [ ]:
list_of_files = readDir("/content/split10_consecutive_average")

In [ ]:
#@title
df_list = []

for item in list_of_files:
    df_list.append(pd.read_csv( item, delimiter = "\t"))

train_df = pd.concat(df_list,ignore_index=False)


In [ ]:
train_df.sample(3)

,doc,par,text,label
1531,www_aftonbladet_se_debatt_article10286052_ab,25,Vill vi leva i ett land där dörren är stängd f...,4.25
2748,z01638_flashback-59138332,39,"Vilka problem (eller ""utmaningar"") ser ni med ...",2.00
2993,z02471_www_aftonbladet_se_debatt_debattamnen_p...,4,När vi talar om flyktingar tänker vi i allmänh...,3.00


In [ ]:
#@title normalize 
import math

label = []

for idx, item in train_df.iterrows():
      lab = math.ceil(item.label)
      label.append(lab)

train_df["label_N"] = label

In [ ]:
train_df.sample(5)

,doc,par,text,label,label_N
3091,z02364_flashback-38429216,1,"25-åriga ""flyktingbarn"" till Laholm.",2.0,2
2109,z03882_flashback-53808922,2,Finland vill alltså ta emot färre flyktingar i...,2.0,2
1703,z02065_www_svd_se_forenklat-att-kalla-avslag-o...,15,Arbetstillstånd är juridiskt sett en förmån. O...,3.0,3
346,z00835_flashback-54930928,7,Kommer svensk media att skriva om detta imorgo...,3.0,3
1303,z00023_www_aftonbladet_se_debatt_a_7kbB4_farre...,14,När alla kommuner delar ansvaret för mottagand...,3.0,3


In [ ]:
#@title sentence model
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, models

model_dir = 'Peltarion/xlm-roberta-longformer-base-4096'
mod = "KB/bert-base-swedish-cased"
sen_xlmr = "sentence-transformers/paraphrase-xlm-r-multilingual-v1"

word_embedding_model = models.Transformer(mod, tokenizer_name_or_path= mod , max_seq_length=512)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

     |████████████████████████████████| 79 kB 7.3 MB/s 
     |████████████████████████████████| 4.2 MB 34.7 MB/s 
     |████████████████████████████████| 1.2 MB 54.3 MB/s 
     |████████████████████████████████| 84 kB 4.1 MB/s 
     |████████████████████████████████| 6.6 MB 53.4 MB/s 
     |████████████████████████████████| 596 kB 66.8 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=bc12ca5590a7bbfb5421a1487ea73336d2e3e6346a1294f82b9a5412ccbea945
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Downloading:   0%|          | 0.00/491 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at KB/bert-base-swedish-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/390k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

#Test

In [ ]:
#@title
#getting all test files

list_of_files = readDir("/content/split10_consecutive_average",split="test")

df_list = []

for item in list_of_files:
    df_list.append(pd.read_csv( item, delimiter = "\t"))

test_df = pd.concat(df_list,ignore_index=False)



In [ ]:
#@title
label = []

for idx, item in test_df.iterrows():
      lab = math.ceil(item.label)
      label.append(lab)

test_df["label_N"] = label

#EDA

In [ ]:
#@title
!git clone https://github.com/mosh98/swe_aug.git
! pip install -r /content/swe_aug/reqs.txt
!wget https://www.ida.liu.se/divisions/hcs/nlplab/swectors/swectors-300dim.txt.bz2
!bzip2 -dk /content/swectors-300dim.txt.bz2
word_vec_path = '/content/swectors-300dim.txt'
from swe_aug import EDA
aug = EDA.Enkel_Data_Augmentation(word_vec_path)

Cloning into 'swe_aug'...
remote: Enumerating objects: 118, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 118 (delta 61), reused 89 (delta 32), pack-reused 0
Receiving objects: 100% (118/118), 516.24 KiB | 13.95 MiB/s, done.
Resolving deltas: 100% (61/61), done.
     |████████████████████████████████| 1.5 MB 778 kB/s 
     |████████████████████████████████| 6.0 MB 56.6 MB/s 
     |████████████████████████████████| 24.2 MB 2.0 MB/s 
     |████████████████████████████████| 749 kB 50.0 MB/s 
     |████████████████████████████████| 457 kB 60.4 MB/s 
     |████████████████████████████████| 10.1 MB 47.6 MB/s 
     |████████████████████████████████| 42 kB 1.8 MB/s 
     |████████████████████████████████| 653 kB 58.1 MB/s 
     |████████████████████████████████| 181 kB 53.4 MB/s 
     |████████████████████████████████| 304 kB 55.1 MB/s 
     |████████████████████████████████| 58 kB 7.2 MB/s 
  Created wheel for ufal.udpipe:

--2022-05-17 17:09:06--  https://www.ida.liu.se/divisions/hcs/nlplab/swectors/swectors-300dim.txt.bz2
Resolving www.ida.liu.se (www.ida.liu.se)... 130.236.57.131, 2001:6b0:17:2004::57:131
Connecting to www.ida.liu.se (www.ida.liu.se)|130.236.57.131|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 195655746 (187M) [application/x-bzip2]
Saving to: ‘swectors-300dim.txt.bz2’

swectors-300dim.txt 100%[===================>] 186.59M  7.26MB/s    in 16s     

2022-05-17 17:09:21 (12.0 MB/s) - ‘swectors-300dim.txt.bz2’ saved [195655746/195655746]

Downloaded pre-trained UDPipe model for 'sv' language
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Fraction of Dataset


In [ ]:
#@title
from sklearn.model_selection import train_test_split


def frac(dataframe, fraction,label):
    """Returns fraction of data"""
    if fraction == 1.0:
      return dataframe
      
    y = dataframe[label]
    train, test = train_test_split(dataframe,train_size=fraction,stratify = y)
    
    return train

def encode_df(dataframe, col = "text"):
  encoded = []
  for idx, item in dataframe.iterrows():
         encoded.append(model.encode(item[col])) 

  return encoded


reports = [] #clf_report (Before Augment, After Augment)


In [ ]:
#@title
test_embed = []

y_test = test_df.label_N

for idx, item in test_df.iterrows():
  test_embed.append(model.encode(item.text))
  

In [ ]:
#@title
from swe_aug.Other_Techniques import Type_SR
aug_t = Type_SR.type_DA(word_vec_path)

Already downloaded a model for the 'sv' language
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#@title
train_df

,doc,par,text,label,label_N
0,www_aftonbladet_se_debatt_article22871619_ab,5,Att vara intresserad av hemländernas politik ä...,3.000000,3
1,www_aftonbladet_se_debatt_article22871619_ab,6,Tyvärr verkar det som om några partier har bli...,2.666667,3
2,www_aftonbladet_se_debatt_article22871619_ab,7,Naturligtvis menar vi inte att politiker med u...,3.000000,3
3,www_aftonbladet_se_debatt_article22871619_ab,8,Vi själva uppfostrades med en ideologi som vi ...,3.666667,4
4,www_aftonbladet_se_debatt_article22871619_ab,9,Det gäller att vara öppen för förändring – men...,3.250000,4
...,...,...,...,...,...
3861,z02107_flashback-15247839,4,"Nu var de, enligt dem själva, kristna också = ...",3.000000,3
3862,z02514_flashback-54883701,1,Professor varnar: bostäderna snart slut mestad...,2.000000,2
3863,z02514_flashback-54883701,2,I SvD varnar professor Stellan Lundström att e...,2.000000,2
3864,z02514_flashback-54883701,3,Att det är invandringen som spetsar till läget...,2.000000,2


In [ ]:

from sklearn.linear_model import SGDClassifier

split_percentage = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
label = "label_N"

for percentage in split_percentage:
    print("______________________________________________")
    
    print("Percentage",percentage)
    
    temp_train_df = frac(train_df, percentage/100,label )
    print("Before Augmentation size: ",temp_train_df.shape)

    y_train = temp_train_df.label_N
    
    train_embed = encode_df( temp_train_df )

    logreg = SGDClassifier(max_iter=3000,n_jobs=-1)
    logreg.fit(train_embed, y_train)
    
    y_pred = logreg.predict(test_embed)
    
    r_1 = classification_report(y_test, y_pred)
    print("Before Augmentation")
    print(r_1)
    print(" ")
    
    #Augmentation EDA
    aug_samples = []
    for idx, item in temp_train_df.iterrows():
      txt = item.text
      lab = item.label_N
      list_of_augs = aug.enkel_augmentation(txt, alpha_sr=0.2, alpha_ri=0.0, alpha_rs=0.0, alpha_rd=0.0, num_aug=2)
      
      for element in list_of_augs:
          aug_samples.append({"text":element,"label_N":lab})

    
    new_aug_EDA_samples = pd.DataFrame(aug_samples)

    
    new_df = pd.concat([temp_train_df,new_aug_EDA_samples], ignore_index=True)
    
    print("After Augmentation size: ",new_df.shape[0])
    
    #augmented train test

    #encode
    train_embed_ = encode_df( new_df)
    
    y_train_ = new_df.label_N
    
    
    sgd_clf_2 = SGDClassifier(max_iter=3000,n_jobs=-1)
    sgd_clf_2.fit(train_embed_, y_train_)

    y_pred = sgd_clf_2.predict(test_embed)


    r_2 = classification_report(y_test, y_pred)
    print("After Augmentation")
    print(r_2)

    #save the reports
    reports.append((percentage, r_1,r_2))



    print("______________________________________________")









______________________________________________
Percentage 10
Before Augmentation size:  (3864, 5)
Before Augmentation
              precision    recall  f1-score   support

           1       0.82      0.31      0.45       440
           2       0.57      0.59      0.58      1127
           3       0.72      0.69      0.70      2239
           4       0.51      0.69      0.59       849
           5       0.50      0.53      0.51       175

    accuracy                           0.63      4830
   macro avg       0.62      0.56      0.57      4830
weighted avg       0.65      0.63      0.62      4830

 
After Augmentation size:  7728
After Augmentation
              precision    recall  f1-score   support

           1       0.74      0.30      0.43       440
           2       0.52      0.66      0.58      1127
           3       0.80      0.44      0.57      2239
           4       0.39      0.83      0.53       849
           5       0.52      0.52      0.52       175

    accuracy   